# Get Item Price History

### Alec Tiefenthal


In [2]:
import requests;                                 # import package used to downlaod data from a web URL
import json;
import pandas as pd;
import matplotlib.pyplot as plt;
import matplotlib;
import datetime as dt
import time
import numpy as np

from pandas.io.json import json_normalize

matplotlib.style.use('ggplot');
%matplotlib inline

In [2]:
dfNew = pd.read_csv('itemOut.csv')
dfNew = dfNew.drop('Unnamed: 0', axis=1) #This may not exist. Check for error.
dfNew = dfNew.drop('Currency', axis=1)
dfNew = dfNew.drop('Value', axis=1)
dfNew = dfNew.drop('Last_Update', axis=1)
dfNew = dfNew.drop('Difference', axis=1)

In [3]:
dfNew.head(5)

,Name,DefIndex,QualInt,Trade,Craft,PriceIndex
0,Dillinger's Duffel,781,6,Tradable,Non-Craftable,0
1,Dillinger's Duffel,781,6,Tradable,Craftable,0
2,Festive Shotgun,1141,11,Tradable,Craftable,0
3,Festive Shotgun,1141,6,Tradable,Craftable,0
4,Carbonado Botkiller Knife Mk.I,901,11,Tradable,Craftable,0


In [4]:
dataf = pd.DataFrame(columns=['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex',
                             'currency', 'timestamp', 'value', 'value_high', 'DateTime'])

In [7]:
dataf

,Name,DefIndex,QualInt,Trade,Craft,PriceIndex,currency,timestamp,value,value_high,DateTime


In [8]:
with open('API Keys.txt', 'r') as keyfile:
    apiKeys=keyfile.read().split('\n')

In [10]:
baseurl = 'http://backpack.tf/api/IGetPriceHistory/v1/?key=' + apiKeys[2]
url = ''

In [60]:
k = 0
i = 0
for index, row in dfNew.iterrows():
    if k > 50:
        print i
        k = 0;
#         break;
    k+=1
    i+=1
#     print k
#     print row
    url = baseurl + '&item=' + repr(int(row.DefIndex))
    url = url + '&quality=' + repr(row.QualInt)
    url = url + '&tradable=' + row.Trade
    url = url + '&cratable=' + row.Craft
    url = url + '&priceindex=' + row.PriceIndex
#     print url
#     time.sleep(0.5)
    data_stream = requests.get(url,stream=True);
    
    # Account for server not responding. Try again.
    while data_stream.status_code == 522:
        url = baseurl + '&item=' + repr(int(row.DefIndex))
        url = url + '&quality=' + repr(int(row.QualInt))
        url = url + '&tradable=' + row.Trade
        url = url + '&cratable=' + row.Craft
        url = url + '&priceindex=' + row.PriceIndex
        time.sleep(0.5)
        data_stream = requests.get(url,stream=True);
    
    rec = data_stream.iter_lines().next().strip()
#     if
    data = json.loads(rec)
    if len(data['response']['history']) == 0:
        continue
#     print url
#     print data
    if data['response']['success'] == 0:
        continue
    
    
#     print data
#     print url
#     print '\n'
    historydf = json_normalize(data['response']['history'])
    historydf['DateTime'] = pd.to_datetime(historydf['timestamp'], unit='s')
    historydf['Name'] = row.Name
    historydf['DefIndex'] = row.DefIndex
    historydf['QualInt'] = row.QualInt
    historydf['Trade'] = row.Trade
    historydf['Craft'] = row.Craft
    historydf['PriceIndex'] = row.PriceIndex
    dataf = pd.concat([dataf, historydf])

51
102
153
204
255
306
357
408
459
510
561
612
663
714
765
816
867
918
969
1020
1071
1122
1173
1224
1275
1326
1377
1428
1479
1530
1581
1632
1683
1734
1785
1836
1887
1938
1989
2040
2091
2142
2193
2244
2295
2346
2397
2448
2499
2550
2601
2652
2703
2754
2805
2856
2907
2958
3009
3060
3111
3162
3213
3264
3315
3366
3417
3468
3519
3570
3621
3672
3723
3774
3825
3876
3927
3978
4029
4080
4131
4182
4233
4284
4335
4386
4437
4488
4539
4590
4641
4692
4743
4794
4845
4896
4947
4998
5049
5100
5151
5202
5253
5304
5355
5406
5457
5508
5559
5610
5661
5712
5763
5814
5865
5916
5967
6018
6069
6120
6171
6222
6273
6324
6375
6426
6477
6528
6579
6630
6681
6732
6783
6834
6885
6936
6987
7038
7089
7140
7191
7242
7293
7344
7395
7446
7497
7548
7599
7650
7701
7752
7803
7854
7905
7956
8007
8058
8109
8160
8211
8262
8313
8364
8415
8466
8517
8568
8619
8670
8721
8772
8823
8874
8925
8976
9027
9078
9129
9180
9231
9282
9333
9384
9435
9486
9537
9588
9639
9690
9741
9792
9843
9894
9945
9996
10047
10098
10149
10200
10251
10302
1035

In [64]:
dataf.head()

,Craft,DateTime,DefIndex,Name,PriceIndex,QualInt,Trade,currency,timestamp,value,value_high
0,Non-Craftable,2012-06-29 14:44:48,781,Dillinger's Duffel,0,6,Tradable,metal,1340981088,5.00,5.00
1,Non-Craftable,2012-07-03 14:22:44,781,Dillinger's Duffel,0,6,Tradable,keys,1341325364,1.00,1.00
2,Non-Craftable,2012-07-09 02:57:41,781,Dillinger's Duffel,0,6,Tradable,metal,1341802661,1.66,1.66
3,Non-Craftable,2012-07-28 21:23:47,781,Dillinger's Duffel,0,6,Tradable,metal,1343510627,1.33,1.33
4,Non-Craftable,2012-09-09 03:48:01,781,Dillinger's Duffel,0,6,Tradable,metal,1347162481,1.66,1.66


In [80]:
st = "hello"
print type(st)

<type 'str'>


In [82]:
dfNew.dtypes

Name           object
DefIndex      float64
QualInt         int64
Trade          object
Craft          object
PriceIndex     object
dtype: object

In [74]:
dataf.dtypes

Craft                 object
DateTime      datetime64[ns]
DefIndex             float64
Name                  object
PriceIndex            object
QualInt              float64
Trade                 object
currency              object
timestamp            float64
value                float64
value_high           float64
dtype: object

In [62]:
finaldataf = dataf
finaldataf = finaldataf.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
finaldataf.sort_index(inplace = True)
#Drop duplicates?
finaldataf.to_csv('itemhistory.csv')

In [3]:
dfIn = pd.read_csv('itemhistory.csv', dtype={'Craft':str, 'DateTime':dt.datetime, 'DefIndex':np.float64, 'Name':str,
                                            'PriceIndex':str, 'QualInt':np.int64, 'Trade':str, 'currency':str,
                                            'timestamp':np.float64, 'value':np.float64, 'value_high':np.float64})
dfIn['DateTime'] = pd.to_datetime(dfIn['DateTime'])
dfIn = dfIn.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'timestamp'])
dfIn.sort_index(inplace = True)

In [4]:
dfIn.head(20)

DateTime  \
Name               DefIndex QualInt Trade    Craft     PriceIndex timestamp                        
A Brush with Death 30186    6       Tradable Craftable 0          1381676945 2013-10-13 15:09:05   
                                                                  1381676945 2013-10-13 15:09:05   
                                                                  1381676945 2013-10-13 15:09:05   
                                                                  1383401806 2013-11-02 14:16:46   
                                                                  1383401806 2013-11-02 14:16:46   
                                                                  1383401806 2013-11-02 14:16:46   
                                                                  1383665506 2013-11-05 15:31:46   
                                                                  1383665506 2013-11-05 15:31:46   
                                                                  1383665506 2013-11-05 15:31:46   
                                                                  1383798437 2013-11-07 04:27:17   
                                                                  1383798437 2013-11-07 04:27:17   
                                                                  1383798437 2013-11-07 04:27:17   
                                                                  1383859373 2013-11-07 21:22:53   
                                                                  1383859373 2013-11-07 21:22:53   
                                                                  1383859373 2013-11-07 21:22:53   
                                                                  1384015717 2013-11-09 16:48:37   
                                                                  1384015717 2013-11-09 16:48:37   
                                                                  1384015717 2013-11-09 16:48:37   
                                                                  1384161954 2013-11-11 09:25:54   
                                                                  1384161954 2013-11-11 09:25:54   

                                                                             currency  \
Name               DefIndex QualInt Trade    Craft     PriceIndex timestamp             
A Brush with Death 30186    6       Tradable Craftable 0          1381676945     keys   
                                                                  1381676945     keys   
                                                                  1381676945     keys   
                                                                  1383401806    metal   
                                                                  1383401806    metal   
                                                                  1383401806    metal   
                                                                  1383665506    metal   
                                                                  1383665506    metal   
                                                                  1383665506    metal   
                                                                  1383798437    metal   
                                                                  1383798437    metal   
                                                                  1383798437    metal   
                                                                  1383859373    metal   
                                                                  1383859373    metal   
                                                                  1383859373    metal   
                                                                  1384015717    metal   
                                                                  1384015717    metal   
                                                                  1384015717    metal   
                                                                  1384161954    metal   
                  